# Surrogate Model Testing

This portion will examine how the various surrogate models perform when used to predict a series of 'optimal' points.

Three designs are examined which were deemed optimal, and drawn from the PHYSOR2020 paper [1].
         | k-eff    | Void    | Doppler <br />
Design 1 | 1.001201 | -114.09 | -0.6965 <br />
Design 2 | 1.004500 | -111.65 | -0.7145 <br />
Design 3 | 1.008623 | -117.14 | -0.5975 <br />

To determine the accuracy of the surrogate models, the three designs will be examined with different surrogate models to determine which model most accurately captures the objectives.

In [77]:
import train_surrogate_models as tm
import db_reshape as dbr
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import model_selection

warnings.filterwarnings("ignore")

df = dbr.create_dataframe(r'sfr_db.h5')
var_tot, obj_tot = dbr.reshape_database(r'sfr_db.h5', ['height', 'smear', 'pu_content'], ['keff', 'void_coeff', 'doppler_coeff'])
var_tot, obj_keff = dbr.reshape_database(r'sfr_db.h5', ['height', 'smear', 'pu_content'], ['keff'])
var_tot, obj_void = dbr.reshape_database(r'sfr_db.h5', ['height', 'smear', 'pu_content'], ['void_coeff'])
var_tot, obj_dopp = dbr.reshape_database(r'sfr_db.h5', ['height', 'smear', 'pu_content'], ['doppler_coeff'])

In [86]:
sm = tm.Surrogate_Models()
models = ['lr', 'pr', 'mars', 'gpr', 'ann', 'rf']
name = ['keff', 'void', 'dopp', 'tot']
name = ['tot']
objective = [obj_keff, obj_void, obj_dopp, obj_tot]
objective = [obj_tot]

dict = {}
designs = {'design_1': [61.37, 51.58, 0.7340], 'design_2': [59.72, 50.01, 0.8694], 'design_3': [71.06, 55.77, 0.3536], 'design_4': [60, 75, 0.0]}
design_known = {'design_1': [1.001201, -114.09, -0.6965], 'design_2': [1.004500, -111.65, -0.7145], 'design_3': [1.008623, -117.14, -0.5975], 'design_4': [1.01390, -116.74, -0.5245]}
data_col = ['model', 'height', 'smear', 'pu', 'k-eff_unk', 'k-eff_known', 'diff_keff', 
            'void_unk', 'void_known', 'diff_void', 'dopp_unk', 'dopp_known', 'diff_dopp', 
            'r-squared', 'mse_score', 'hyper_param']

sm_db = pd.DataFrame(columns = data_col)

for design, variables in designs.items():
  for model in models:
    sm.random = 0
    sm.update_database(var_tot, obj_tot)
    sm.update_model(model)
    sm.optimize_model(model)
    score = sm.models[model]['score']
    mse_score = sm.models[model]['mse_score']
    hp = sm.models[model]['hyper_parameters']
    cv = sm.models[model]['cv_results']    
    val = sm.predict(model, [variables])
    keff = val[0][0]
    void = val[0][1]
    dopp = val[0][2]
    append_dict = pd.DataFrame([[model,
                                 designs[design][0], designs[design][1], designs[design][2],
                                 keff, design_known[design][0], (keff - design_known[design][0])/keff*100,
                                 void, design_known[design][1], (void - design_known[design][1])/void*100,
                                 dopp, design_known[design][2], (dopp - design_known[design][2])/dopp*100,                                 
                                 score,
                                 mse_score,
                                 hp]], index = [design], columns=data_col)
    sm_db = sm_db.append(append_dict)
    sm.clear_surrogate_model()
sm_db

,model,height,smear,pu,k-eff_unk,k-eff_known,diff_keff,void_unk,void_known,diff_void,dopp_unk,dopp_known,diff_dopp,r-squared,mse_score,hyper_param
design_1,lr,61.37,51.58,0.7340,0.996083,1.001201,-0.513786,-122.959887,-114.09,7.213643,-0.756249,-0.6965,7.900736,0.877455,0.122545,None
design_1,pr,61.37,51.58,0.7340,1.004680,1.001201,0.346261,-113.968766,-114.09,-0.106374,-0.719297,-0.6965,3.169402,0.789586,0.210414,{'poly__degree': 2}
design_1,mars,61.37,51.58,0.7340,1.007700,1.001201,0.644913,-121.958232,-114.09,6.451580,-0.757533,-0.6965,8.056856,0.850838,0.149162,"{'endspan_alpha': 0.1, 'penalty': 7}"
design_1,gpr,61.37,51.58,0.7340,1.004819,1.001201,0.360027,-111.177912,-114.09,-2.619305,-0.717027,-0.6965,2.862757,0.898651,0.101349,"{'kernel': Matern(length_scale=1, nu=1.5)}"
design_1,ann,61.37,51.58,0.7340,1.004801,1.001201,0.358272,-113.288383,-114.09,-0.707589,-0.713306,-0.6965,2.356076,0.744553,0.255447,"{'activation': 'logistic', 'hidden_layer_sizes..."
design_1,rf,61.37,51.58,0.7340,0.965166,1.001201,-3.733570,-130.708165,-114.09,12.713946,-0.782896,-0.6965,11.035424,0.952333,0.047667,{'n_estimators': 300}
design_2,lr,59.72,50.01,0.8694,1.010240,1.004500,0.568175,-117.003131,-111.65,4.575204,-0.752184,-0.7145,5.009899,0.877455,0.122545,None
design_2,pr,59.72,50.01,0.8694,1.003077,1.004500,-0.141892,-110.879884,-111.65,-0.694550,-0.739165,-0.7145,3.336929,0.789586,0.210414,{'poly__degree': 2}
design_2,mars,59.72,50.01,0.8694,1.011845,1.004500,0.725945,-120.083965,-111.65,7.023390,-0.773623,-0.7145,7.642312,0.850838,0.149162,"{'endspan_alpha': 0.1, 'penalty': 7}"
design_2,gpr,59.72,50.01,0.8694,1.004152,1.004500,-0.034611,-109.131948,-111.65,-2.307346,-0.759373,-0.7145,5.909268,0.898651,0.101349,"{'kernel': Matern(length_scale=1, nu=1.5)}"


In [60]:
df

,beta,capture_fraction,condition,doppler_coeff,enrichment,epithermal_fraction,escape_fraction,fast_fraction,fission_fraction,gen_time,height,keff,nu-bar,smear,temperature,thermal_fraction,void_coeff,pu_content
FC_FS50_H50_15Pu12U10Zr,"[0.00461, 6e-05]",[0.520715],[900K],[-0.9207862603943795],[15Pu12U10Zr],[37.92],[0.154298],[62.07],[0.324987],"[-5.72991e-06, 7.31305e-08]",[50.0],"[0.89335, 4e-05]",[2.745],[50.0],[900.0],[0.01],[-160.66393],[0.55555555]
FC_FS50_H50_27Pu0U10Zr,"[0.00286, 4e-05]",[0.520791],[900K],[-0.7887010724516585],[27Pu0U10Zr],[33.05],[0.152357],[66.94],[0.326852],"[-4.11376e-06, 6.57185e-08]",[50.0],"[0.96535, 4e-05]",[2.95],[50.0],[900.0],[0.01],[-120.20229],[1.0]
FC_FS50_H50_27U10Zr,"[0.0075, 7e-05]",[0.538368],[900K],[-1.1920563338208012],[27U10Zr],[44.56],[0.15849],[55.42],[0.303142],"[-6.52466e-06, 6.86275e-08]",[50.0],"[0.75794, 4e-05]",[2.497],[50.0],[900.0],[0.02],[-254.72565],[0.0]
FC_FS50_H60_15Pu12U10Zr,"[0.0045, 6e-05]",[0.507838],[900K],[-0.8033728590817191],[15Pu12U10Zr],[38.21],[0.147135],[61.78],[0.345027],"[-6.34425e-06, 8.09447e-08]",[60.0],"[0.9478, 5e-05]",[2.744],[50.0],[900.0],[0.01],[-137.7967],[0.55555555]
FC_FS50_H60_27Pu0U10Zr,"[0.00285, 4e-05]",[0.50821],[900K],[-0.7524701231886866],[27Pu0U10Zr],[33.38],[0.145132],[66.61],[0.346657],"[-4.68822e-06, 7.34927e-08]",[60.0],"[1.02345, 4e-05]",[2.949],[50.0],[900.0],[0.01],[-101.41381],[1.0]
FC_FS50_H60_27U10Zr,"[0.00718, 7e-05]",[0.526086],[900K],[-1.1111255844298285],[27U10Zr],[44.84],[0.151447],[55.14],[0.322467],"[-7.11677e-06, 7.56834e-08]",[60.0],"[0.80575, 4e-05]",[2.496],[50.0],[900.0],[0.02],[-221.64342],[0.0]
FC_FS50_H70_15Pu12U10Zr,"[0.00452, 5e-05]",[0.497209],[900K],[-0.7346642178163165],[15Pu12U10Zr],[38.5],[0.141822],[61.49],[0.360969],"[-7.04972e-06, 8.83638e-08]",[70.0],"[0.99118, 4e-05]",[2.742],[50.0],[900.0],[0.01],[-120.82122],[0.55555555]
FC_FS50_H70_27Pu0U10Zr,"[0.00279, 4e-05]",[0.49774],[900K],[-0.6395678610715549],[27Pu0U10Zr],[33.7],[0.139831],[66.29],[0.362429],"[-5.03588e-06, 7.93128e-08]",[70.0],"[1.0697, 4e-05]",[2.948],[50.0],[900.0],[0.01],[-87.772484],[1.0]
FC_FS50_H70_27U10Zr,"[0.00721, 7e-05]",[0.515747],[900K],[-0.9843844054557849],[27U10Zr],[45.15],[0.146223],[54.84],[0.33803],"[-7.91973e-06, 8.28775e-08]",[70.0],"[0.84421, 4e-05]",[2.494],[50.0],[900.0],[0.01],[-196.91116],[0.0]
FC_FS50_H80_15Pu12U10Zr,"[0.00448, 5e-05]",[0.486589],[900K],[-0.7047030067042215],[15Pu12U10Zr],[38.75],[0.139624],[61.24],[0.373787],"[-7.55031e-06, 9.48554e-08]",[80.0],"[1.02594, 4e-05]",[2.741],[50.0],[900.0],[0.01],[-109.992775],[0.55555555]
